In [6]:
from spytial import *
from spytial.annotations import *
from spytial.annotations import flag

In [7]:
# Setup for performance metrics
import random
from time import sleep
perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 30
SIZES = [5, 10, 25, 50]

# Max Heap

In [ ]:
from typing import List

HEAP_VALUES = "((int - 0).(list.idx))"
LEFT_CHILDREN = f"{{ parent, child : {HEAP_VALUES} | (some i, i2 : int | (i->parent + i2->child) in (list.idx) and @num:i2 = multiply[@num:i, 2])}}"

RIGHT_CHILDREN = f"{{ parent, child : {HEAP_VALUES} | (some i, i2 : int | (i->parent + i2->child) in (list.idx) and @num:i2 = add[1, multiply[@num:i, 2]])}}"
print(LEFT_CHILDREN)

## TODO: I think these annotations can be simplified a bit
@hideAtom(selector=f'MaxHeap + list + (int - {HEAP_VALUES})')

@orientation(selector= LEFT_CHILDREN, directions=["left", "below"])
@inferredEdge(selector=LEFT_CHILDREN, name = "left")
@orientation(selector=RIGHT_CHILDREN, directions=["right", "below"])
@inferredEdge(selector=RIGHT_CHILDREN, name = "right")
class MaxHeap:
    """
    CLRS-style max heap storing integers.
    1-indexed: a[0] unused.
    """
    def __init__(self, data: List[int] = None):
        self.a: List[int] = [0]
        if data:
            self.a.extend(data)
        self.n = len(self.a) - 1
        if self.n > 1:
            self.build_max_heap()

    # index helpers
    @staticmethod
    def _parent(i: int) -> int: return i // 2
    @staticmethod
    def _left(i: int) -> int:   return 2 * i
    @staticmethod
    def _right(i: int) -> int:  return 2 * i + 1

    def _max_heapify(self, i: int) -> None:
        while True:
            l, r = self._left(i), self._right(i)
            largest = i
            if l <= self.n and self.a[l] > self.a[largest]:
                largest = l
            if r <= self.n and self.a[r] > self.a[largest]:
                largest = r
            if largest == i:
                break
            self.a[i], self.a[largest] = self.a[largest], self.a[i]
            i = largest

    def build_max_heap(self) -> None:
        for i in range(self.n // 2, 0, -1):
            self._max_heapify(i)

    # API
    def max(self) -> int:
        if self.n < 1:
            raise IndexError("heap underflow")
        return self.a[1]

    def extract_max(self) -> int:
        if self.n < 1:
            raise IndexError("heap underflow")
        m = self.a[1]
        self.a[1] = self.a[self.n]
        self.a.pop()
        self.n -= 1
        if self.n >= 1:
            self._max_heapify(1)
        return m

    def increase_key(self, i: int, key: int) -> None:
        if i < 1 or i > self.n:
            raise IndexError("index out of range")
        if key < self.a[i]:
            raise ValueError("new key is smaller than current key")
        self.a[i] = key
        while i > 1 and self.a[self._parent(i)] < self.a[i]:
            p = self._parent(i)
            self.a[i], self.a[p] = self.a[p], self.a[i]
            i = p

    def insert(self, key: int) -> None:
        self.n += 1
        self.a.append(float("-inf"))  # sentinel
        self.increase_key(self.n, key)

    def __len__(self) -> int:
        return self.n

    def __repr__(self) -> str:
        return f"MaxHeap({self.a[1:]})"





{ parent, child : ((int - 0).(list.idx))) | (some i, i2 : int | (i->parent + i2->child) in (list.idx) and @num:i2 = multiply[@num:i, 2])}


## Max Heap In CLRS

![max-heap](./img/max-heap.png)

In [26]:
h = MaxHeap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
diagram(h)

## Performance - Max Heap

In [10]:
STRUCTURE = "max_heap"
for size in SIZES:
    values = random.sample(range(1, 1000), size)
    h = MaxHeap(values)
    
    print(f"{STRUCTURE}({size} elements): Rendering with perf_iterations={PI}...")
    diagram(h, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI)
    sleep(2)

max_heap(5 elements): Rendering with perf_iterations=30...
max_heap(10 elements): Rendering with perf_iterations=30...
max_heap(25 elements): Rendering with perf_iterations=30...
max_heap(50 elements): Rendering with perf_iterations=30...


# Fibonacci Heap



In [11]:
import math


# Fibonacci Heap (priority queue) with spatial annotations for visualization
@attribute(field="key")
@attribute(field="degree")
@attribute(field="mark")
@atomColor(selector='{ x : FibNode | @bool:(x.mark) }', value='black')
@align(selector="{ r1, r2 : FibNode | (r1 != r2) and (r1.parent = r2.parent) }", direction="horizontal")
@orientation(selector="{ p,c : FibNode |  c->p in parent}", directions=["below"])
class FibNode:
    def __init__(self, key=None):
        self.key = key
        self.degree = 0
        self.mark = False
        self.parent = None
        self.child = None
        # circular doubly-linked list pointers
        self.left = self
        self.right = self


@attribute(field="n")
@hideAtom(selector='NoneType + int + bool')
@orientation(selector='min', directions=['directlyBelow'])
class FibonacciHeap:
    def __init__(self):
        self.min: FibNode | None = None
        self.n: int = 0

    # helper: iterate a circular doubly-linked list starting at node
    def _iterate(self, start: FibNode):
        if start is None:
            return
        node = start
        while True:
            yield node
            node = node.right
            if node is start:
                break

    # insert a new key, return the node
    def insert(self, key):
        x = FibNode(key=key)
        # add to root list
        if self.min is None:
            self.min = x
        else:
            # insert x to the right of min
            x.right = self.min.right
            x.left = self.min
            self.min.right.left = x
            self.min.right = x
            if x.key < self.min.key:
                self.min = x
        self.n += 1
        return x

    def find_min(self):
        return self.min

    # merge another heap into this one (destructive)
    def union(self, other: "FibonacciHeap"):
        if other is None or other.min is None:
            return
        if self.min is None:
            self.min = other.min
            self.n = other.n
            return
        # concatenate root lists
        a = self.min.right
        b = other.min.left
        self.min.right = other.min
        other.min.left = self.min
        a.left = b
        b.right = a
        if other.min.key < self.min.key:
            self.min = other.min
        self.n += other.n

    # extract the minimum node and return it
    def extract_min(self):
        z = self.min
        if z is not None:
            # move z's children to root list
            if z.child is not None:
                children = list(self._iterate(z.child))
                for x in children:
                    # remove parent link
                    x.parent = None
                    # splice x into root list (to the right of min)
                    x.left = self.min
                    x.right = self.min.right
                    self.min.right.left = x
                    self.min.right = x
            # remove z from root list
            if z is z.right:
                self.min = None
            else:
                z.left.right = z.right
                z.right.left = z.left
                self.min = z.right
                self._consolidate()
            self.n -= 1
        return z

    def _consolidate(self):
        if self.min is None:
            return
        # upper bound on degree
        max_degree = int(math.log(self.n, 2)) + 2 if self.n > 0 else 1
        A = [None] * (max_degree + 1)
        roots = list(self._iterate(self.min))
        for w in roots:
            x = w
            d = x.degree
            while A[d] is not None:
                y = A[d]
                if x.key > y.key:
                    x, y = y, x
                self._link(y, x)
                A[d] = None
                d += 1
            A[d] = x
        # rebuild root list and find new min
        self.min = None
        for node in A:
            if node is not None:
                # isolate node
                node.left = node.right = node
                if self.min is None:
                    self.min = node
                else:
                    # insert to root list
                    node.right = self.min.right
                    node.left = self.min
                    self.min.right.left = node
                    self.min.right = node
                    if node.key < self.min.key:
                        self.min = node

    def _link(self, y: FibNode, x: FibNode):
        # remove y from root list
        y.left.right = y.right
        y.right.left = y.left
        # make y a child of x
        y.parent = x
        y.left = y.right = y
        if x.child is None:
            x.child = y
        else:
            # insert y into x's child circular list
            y.right = x.child.right
            y.left = x.child
            x.child.right.left = y
            x.child.right = y
        x.degree += 1
        y.mark = False

    def decrease_key(self, x: FibNode, k):
        if k > x.key:
            raise ValueError("new key is greater than current key")
        x.key = k
        y = x.parent
        if y is not None and x.key < y.key:
            self._cut(x, y)
            self._cascading_cut(y)
        if x.key < self.min.key:
            self.min = x

    def _cut(self, x: FibNode, y: FibNode):
        # remove x from y's child list
        if y.child is x:
            # if x is the only child
            if x.right is x:
                y.child = None
            else:
                y.child = x.right
        x.left.right = x.right
        x.right.left = x.left
        y.degree -= 1
        # add x to root list
        x.left = self.min
        x.right = self.min.right
        self.min.right.left = x
        self.min.right = x
        x.parent = None
        x.mark = False

    def _cascading_cut(self, y: FibNode):
        z = y.parent
        if z is not None:
            if not y.mark:
                y.mark = True
            else:
                self._cut(y, z)
                self._cascading_cut(z)

    def delete(self, x: FibNode):
        # decrease key to -infinity then extract min
        self.decrease_key(x, -float("inf"))
        self.extract_min()


![fh.png](./img/fibonacci-heap.png)

In [12]:
H = FibonacciHeap()

# --- seed the five final roots as plain roots (no consolidation yet) ---
n23 = H.insert(23)
n7  = H.insert(7)
n3  = H.insert(3)      # will be min
n17 = H.insert(17)
n24 = H.insert(24)

# --- stage helpers to build degrees for consolidation pairings ---
n18 = H.insert(18)
n52 = H.insert(52)
n38 = H.insert(38)
n39 = H.insert(39)
n152 = H.insert(152)   # helper child for 52
n138 = H.insert(138)   # helper child for 38

# 1) Consolidate once to create three degree-1 roots:
#    18<-39, 52<-152, 38<-138
H.insert(1); H.extract_min()   # remove 1, consolidate

# 2) Raise 3 to degree 1, then collide 3 with 18 (both deg=1) so 18 becomes child of 3.
#    Then collide 3 (now deg=2) with 52 (deg=1) via an extra deg-0 step in the same round,
#    ending with 3 having children {18,52}.
H.insert(2); H.extract_min()   # remove 2, consolidate

# 3) Give 38 degree 2, then collide 3 (deg=2) with 38 (deg=2) so 38 becomes child of 3.
H.insert(4); H.extract_min()   # remove 4, consolidate

# Add a temporary second child under 18, then cut it once.
tmp = H.insert(50)        # becomes a root
H._link(tmp, n18)         # tmp is now a child of 18  (degree(18) increases)
H.decrease_key(tmp, -1)   # cut tmp to root; FIRST loss -> sets n18.mark = True


diagram(H)

## Performance

In [13]:
STRUCTURE = "fibonacci_heap"
for size in SIZES:
    H = FibonacciHeap()
    values = random.sample(range(1, 1000), size)
    
    # Insert all values
    nodes = [H.insert(val) for val in values]
    
    # Perform some operations to create interesting structure
    if size > 2:
        # Extract min to trigger consolidation
        H.extract_min()
        # Do some decrease_key operations to create cuts
        for i in range(min(3, len(nodes) - 1)):
            if nodes[i].key > 10:
                H.decrease_key(nodes[i], nodes[i].key - 5)
    
    print(f"{STRUCTURE}({size} elements): Rendering with perf_iterations={PI}...")
    diagram(H, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI)
    sleep(2)



fibonacci_heap(5 elements): Rendering with perf_iterations=30...
fibonacci_heap(10 elements): Rendering with perf_iterations=30...
fibonacci_heap(25 elements): Rendering with perf_iterations=30...
fibonacci_heap(50 elements): Rendering with perf_iterations=30...
